In [14]:
from pymongo import MongoClient
import pandas as pd

# Load the original CSV file
csv_path = '/Users/nicholasnoto/Desktop/CIS 4400/Homework1/Mental_Health_Care_in_the_Last_4_Weeks.csv'
original_data = pd.read_csv(csv_path)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['mental_health']

# Function to insert data into MongoDB with a type field
def insert_with_type(dataframe, collection_name, type_name):
    dataframe['type'] = type_name
    collection = db[collection_name]
    collection.insert_many(dataframe.to_dict('records'))
    print(f"{type_name} data inserted into {collection_name} collection.")

# Insert the original data into FactMentalHealth collection
insert_with_type(original_data, 'FactMentalHealth', 'FactMentalHealth')

# Load data from MongoDB
fact_collection = db['FactMentalHealth']
data = pd.DataFrame(list(fact_collection.find()))
print("Data loaded from MongoDB:")
print(data.head())

# Drop _id column if present
data.drop(columns=['_id'], inplace=True, errors='ignore')

# Unified date format and split into multiple units
data['Time Period Start Date'] = pd.to_datetime(data['Time Period Start Date'], errors='coerce')
data['Time Period End Date'] = pd.to_datetime(data['Time Period End Date'], errors='coerce')

data['Start Year'] = data['Time Period Start Date'].dt.year
data['Start Quarter'] = data['Time Period Start Date'].dt.quarter
data['Start Month'] = data['Time Period Start Date'].dt.month
data['Start Day'] = data['Time Period Start Date'].dt.day

print("Data after date transformations:")
print(data[['Time Period Start Date', 'Time Period End Date', 'Start Year', 'Start Quarter', 'Start Month', 'Start Day']].head())

# Handle NULL values
data.drop(columns=['Suppression Flag'], inplace=True)  # Drop the Suppression Flag column
data['Quartile Range'].fillna('Unknown', inplace=True)  # Fill NULL values in Quartile Range with 'Unknown'

# Remove duplicate rows if necessary
data.drop_duplicates(inplace=True)
print("Data count after dropping duplicates:", len(data))

# Verify data against data reference (example for State)
valid_states = ["United States"]  # Update this list as per your requirement
data = data[data['State'].isin(valid_states)]
print("Data count after verifying states:", len(data))

# Add new columns and perform calculations
data['CI_Sum'] = data['LowCI'] + data['HighCI']
print("Data with new column CI_Sum:")
print(data[['LowCI', 'HighCI', 'CI_Sum']].head())

# Print dataframes to check before insertion
print("DimIndicator DataFrame:")
indicator_data = data[['Indicator']].drop_duplicates().dropna().reset_index(drop=True)
print(indicator_data)

print("DimTime DataFrame:")
time_data = data[['Time Period', 'Time Period Label', 'Time Period Start Date', 'Time Period End Date', 'Start Year', 'Start Quarter', 'Start Month', 'Start Day']].drop_duplicates().dropna().reset_index(drop=True)
print(time_data)

print("DimLocation DataFrame:")
location_data = data[['State']].drop_duplicates().dropna().reset_index(drop=True)
print(location_data)

print("FactMentalHealth DataFrame:")
fact_data = data.copy()
print(fact_data.head())

# Function to update data in MongoDB with a type field
def update_with_type(dataframe, collection_name, type_name):
    collection = db[collection_name]
    collection.delete_many({'type': type_name})  # Clear existing data of this type
    if not dataframe.empty:
        dataframe['type'] = type_name
        collection.insert_many(dataframe.to_dict('records'))
        print(f"{type_name} data updated in {collection_name} collection.")
    else:
        print(f"{type_name} DataFrame is empty. No data updated in {collection_name} collection.")

# Update DimIndicator
update_with_type(indicator_data, 'DimIndicator', 'DimIndicator')

# Update DimTime
update_with_type(time_data, 'DimTime', 'DimTime')

# Update DimLocation
update_with_type(location_data, 'DimLocation', 'DimLocation')

# Update FactMentalHealth
update_with_type(fact_data, 'FactMentalHealth', 'FactMentalHealth')


FactMentalHealth data inserted into FactMentalHealth collection.
Data loaded from MongoDB:
                        _id  \
0  6644cf93301ef9cf8641d064   
1  6644cf93301ef9cf8641d065   
2  6644cf93301ef9cf8641d066   
3  6644cf93301ef9cf8641d067   
4  6644cf93301ef9cf8641d068   

                                           Indicator              Group  \
0  Took Prescription Medication for Mental Health...  National Estimate   
1  Took Prescription Medication for Mental Health...             By Age   
2  Took Prescription Medication for Mental Health...             By Age   
3  Took Prescription Medication for Mental Health...             By Age   
4  Took Prescription Medication for Mental Health...             By Age   

           State       Subgroup Phase  Time Period      Time Period Label  \
0  United States  United States     2           13  Aug 19 - Aug 31, 2020   
1  United States  18 - 29 years     2           13  Aug 19 - Aug 31, 2020   
2  United States  30 - 39 years     2   

FactMentalHealth data updated in FactMentalHealth collection.
